In [1]:
import numpy as np
import pandas as pd
import random
import plotly.express as px
from datetime import datetime

rows_to_keep = 43

sheet_data = pd.read_excel("https://docs.google.com/spreadsheets/d/1DuYUj2ODS8D3PWK42ZopUD1dqcg89ckI6vPn71LidGo/export?format=xlsx")
sheet_data = sheet_data.iloc[:rows_to_keep].drop(columns=["Overall % Lost", "Imgur"]).dropna(axis=1, how="all")
sheet_data = sheet_data.rename(columns={"$": "Pot Contribution", "Starting 1/1/22": "Starting Weight"})
sheet_data = sheet_data.melt(id_vars=["Participant", "Pot Contribution", "Paid?", "Starting Weight"], var_name="Date", value_name="Weight")
sheet_data["Starting Weight"] = pd.to_numeric(sheet_data["Starting Weight"])
sheet_data["Weight"] = pd.to_numeric(sheet_data["Weight"])
sheet_data = sheet_data.dropna(subset=["Weight", "Starting Weight"]).sort_values(by=["Participant", "Date"]).reset_index()

by_participant = sheet_data.groupby("Participant")

sheet_data["Previous Week Weight"] = by_participant["Weight"].shift(1)
sheet_data.loc[sheet_data["Previous Week Weight"].isnull(), "Previous Week Weight"] = sheet_data.loc[sheet_data["Previous Week Weight"].isnull(), "Starting Weight"]
sheet_data["Weight Difference"] = sheet_data["Weight"] - sheet_data["Previous Week Weight"]

sheet_data["Cumulative Weight Lost"] = by_participant["Weight Difference"].transform(pd.Series.cumsum)
sheet_data["Cumulative % Lost"] = sheet_data["Cumulative Weight Lost"] / sheet_data["Starting Weight"] * -100
sheet_data["Participant Index"] = by_participant.ngroup()

participant_colors = ["#%06x" % random.randint(0, 0xFFFFFF) for participant, _ in by_participant]


In [13]:
total_weight_lost = pd.DataFrame(sheet_data.groupby("Date")["Weight Difference"].sum()).reset_index()
total_weight_lost["Total Weight Lost"] = total_weight_lost["Weight Difference"].cumsum()

fig = px.line(total_weight_lost, x="Date", y="Total Weight Lost", title="Total Weight Lost")
fig.update_xaxes(range=[datetime(2022,1,1), datetime(2022, 4, 22)])
fig.update_yaxes(autorange="reversed")
fig.show()

In [2]:
fig = px.line(sheet_data, x="Date", y="Weight", color='Participant', title="Weight over time")
fig.update_xaxes(range=[datetime(2022,1,1), datetime(2022, 4, 22)])
fig.update_yaxes(range=[130, 400])
fig.show()

In [3]:
fig = px.bar(sheet_data, x="Date", y="Weight Difference", color='Participant', title="Weight Difference by Week", labels={"Weight Difference":"Weight Difference (lbs)"})
fig.update_xaxes(range=[datetime(2022,1,1), datetime(2022, 4, 22)])
fig.update_yaxes(autorange="reversed")
fig.show()

In [6]:
fig = px.line(sheet_data, x="Date", y="Cumulative % Lost", color='Participant', title="Cumulative % Lost")
fig.update_xaxes(range=[datetime(2022,1,1), datetime(2022, 4, 22)])
fig.show()